In [74]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [75]:
def gap(row):
    if abs(row['dualBound']) > 999999999 or abs(row['primalBound']) > 999999999 or \
            row['dualBound'] * row['primalBound'] < 0:
        return 1
    else:
        num = abs(row['primalBound'] - row['dualBound'])
        den = max(abs(row['primalBound']), abs(row['dualBound']))
        return num/den

In [76]:
def infeasible(row):
    if abs(row['primalBound']) > 999999999:
        return 1
    else:
        return 0

In [77]:
def make_stats(directory, vpc=False):
    dfs = {}
    stats = []
    batch_stats = []
    stems = []
    for csv in os.listdir(directory):
        stem = csv.split('.')[0]
        stems.append(stem)
        df = pd.read_csv(os.path.join(directory, csv))
        if vpc:
            df['rootDualBoundTime'] += (df['completionTime'] - df['terminationTime'])  # add time spent making VPCs
        df['primalBound'] = pd.to_numeric(df['primalBound'], errors='coerce')
        df['heuristicPrimalBound'] = pd.to_numeric(df['heuristicPrimalBound'], errors='coerce')
        df['reltime'] = df['completionTime'] / df['maxCompletionTime']
        df['gap'] = df.apply(lambda row: gap(row), axis=1)
        df['nofeas'] = df.apply(lambda row: infeasible(row), axis=1)
        df['batch'] = np.array(df.index)//10 + 1
        stats.append(df[['reltime', 'gap', 'nofeas']].mean())
        df_batch = df.groupby('batch').mean()[['reltime', 'gap', 'nofeas']].reset_index()
        df_batch['instance set'] = stem
        batch_stats.append(df_batch)
        dfs[stem] = df
    df = pd.DataFrame(stats)
    df['instance set'] = stems
    df = df[['instance set', 'reltime', 'gap', 'nofeas']]
    df = df.set_index('instance set')
    df.index.name = None
    
    df_batch = pd.concat(batch_stats)
    df_batch = df_batch[['instance set', 'batch', 'reltime', 'gap', 'nofeas']]
    df_batch = df_batch.set_index(['instance set', 'batch'])
    df_batch.index.name = None
    
    return df, dfs, df_batch

In [78]:
def combine(bdf, vdf):
    # join the two dataframes on their index and combine the columns into a multiindex
    return pd.concat([bdf, vdf], axis=1, keys=["without VPCs", "with VPCs"]).swaplevel(0, 1, axis=1).sort_index(axis=1)

In [79]:
baseline_df, baseline_dfs, baseline_batch = make_stats('baseline', vpc=False)
vpc_df, vpc_dfs, vpc_batch = make_stats('vpcs', vpc=True)
compare = combine(baseline_df, vpc_df)
compare_batch = combine(baseline_batch, vpc_batch)

In [80]:
compare.to_csv('compare.csv', float_format='%.3f')
compare_batch.to_csv('compare_batch.csv', float_format='%.3f')

In [81]:
compare

gap                 nofeas               \
                         with VPCs without VPCs with VPCs without VPCs   
rhs_series_1              0.866624     0.053974  0.800000         0.00   
rhs_series_2              0.005101     0.005101  0.000000         0.00   
matrix_series_1           0.780883     0.786396  0.000000         0.00   
mat_rhs_bnd_obj_series_1  0.449548     0.161656  0.320000         0.08   
obj_series_1              0.009635     0.006335  0.000000         0.00   
rhs_obj_series_1          1.000000     0.159973  1.000000         0.14   
obj_series_2              0.972149     0.970676  0.833333         0.78   
bnd_series_2              1.000000     1.000000  1.000000         1.00   
bnd_series_1              0.457293     0.418395  0.000000         0.00   

                           reltime               
                         with VPCs without VPCs  
rhs_series_1              1.079750     1.005000  
rhs_series_2              3.261000     3.087333  
matrix_series_1           1.020733     1.022733  
mat_rhs_bnd_obj_series_1  1.003444     1.008000  
obj_series_1              0.990900     1.000550  
rhs_obj_series_1          5.005600     1.212840  
obj_series_2              1.009667     1.026000  
bnd_series_2              0.989733     1.003933  
bnd_series_1              1.006486     1.001000

In [82]:
compare_batch

gap                 nofeas               \
                               with VPCs without VPCs with VPCs without VPCs   
instance set             batch                                                 
bnd_series_1             1      0.475544     0.410803       0.0          0.0   
                         2      0.465385     0.443894       0.0          0.0   
                         3      0.414930     0.425997       0.0          0.0   
                         4      0.480178     0.408387       0.0          0.0   
                         5           NaN     0.402894       NaN          0.0   
bnd_series_2             1      1.000000     1.000000       1.0          1.0   
                         2      1.000000     1.000000       1.0          1.0   
                         3      1.000000     1.000000       1.0          1.0   
                         4      1.000000     1.000000       1.0          1.0   
                         5      1.000000     1.000000       1.0          1.0   
mat_rhs_bnd_obj_series_1 1      0.400498     0.204845       0.3          0.1   
                         2      0.522210     0.142068       0.4          0.1   
                         3      0.721655     0.156478       0.6          0.1   
                         4      0.191580     0.112554       0.0          0.0   
                         5      0.411795     0.192336       0.3          0.1   
matrix_series_1          1      0.786244     0.785509       0.0          0.0   
                         2      0.780871     0.787701       0.0          0.0   
                         3      0.776623     0.786203       0.0          0.0   
                         4      0.782184     0.786586       0.0          0.0   
                         5      0.778492     0.785980       0.0          0.0   
obj_series_1             1      0.006999     0.005600       0.0          0.0   
                         2      0.010419     0.006617       0.0          0.0   
                         3      0.009581     0.006442       0.0          0.0   
                         4      0.010555     0.006598       0.0          0.0   
                         5      0.010624     0.006417       0.0          0.0   
obj_series_2             1      1.000000     0.853379       0.9          0.8   
                         2      1.000000     1.000000       0.7          0.9   
                         3      0.916446     1.000000       0.9          0.8   
                         4           NaN     1.000000       NaN          0.7   
                         5           NaN     1.000000       NaN          0.7   
rhs_obj_series_1         1      1.000000     0.206138       1.0          0.2   
                         2           NaN     0.358978       NaN          0.3   
                         3           NaN     0.115510       NaN          0.1   
                         4           NaN     0.111426       NaN          0.1   
                         5           NaN     0.007813       NaN          0.0   
rhs_series_1             1      0.333122     0.048140       0.0          0.0   
                         2      1.000000     0.057582       1.0          0.0   
                         3      1.000000     0.047962       1.0          0.0   
                         4      1.000000     0.071238       1.0          0.0   
                         5      1.000000     0.044947       1.0          0.0   
rhs_series_2             1      0.005232     0.005232       0.0          0.0   
                         2      0.005042     0.005042       0.0          0.0   
                         3      0.004999     0.004999       0.0          0.0   
                         4      0.005039     0.005039       0.0          0.0   
                         5      0.005193     0.005193       0.0          0.0   

                                 reltime               
                               with VPCs without VPCs  
instance set             batch                         
bnd_series_1             1  

In [85]:
# for each instance in compare.index, plot baseline_dfs[rootDualBound] on its own plot
for instance in compare.index:
    fig=plt.figure()
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212, sharex = ax1)

    ax1.plot(baseline_dfs[instance]['dualBound'], label='without VPCs')
    ax1.plot(vpc_dfs[instance]['dualBound'], label='with VPCs')
    ax1.axvline(x=10, color='black', label='begin parametric VPCs')
    # ax1.legend(loc='upper right')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax1.title.set_text(f'{instance}\nDual Bound Comparison with and without VPCs')
    ax1.set_ylabel('Dual Bound')

    ax2.plot(baseline_dfs[instance]['rootDualBoundTime'], label='without VPCs')
    ax2.plot(vpc_dfs[instance]['rootDualBoundTime'], label='with VPCs')
    ax2.axvline(x=10, color='black', label='begin parametric VPCs')
    # ax2.legend(loc='upper right')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax2.title.set_text(f'{instance}\nRoot Cut Generation Time with and without VPCs')
    ax2.set_xlabel('Instance Number')
    ax2.set_ylabel('Seconds')

    fig.tight_layout()
    plt.savefig(f'{instance}_dual_bounds.png')
    plt.close()

In [69]:
# bound series 1 and matrix series 1
vpc_dfs['bnd_series_1']

,lpBound,rootDualBound,dualBound,heuristicPrimalBound,primalBound,heuristicTime,rootDualBoundTime,terminationTime,maxTerminationTime,completionTime,maxCompletionTime,usePreprocessing,vpcGenerator,reltime,gap,nofeas,batch
0,7830.336037,10594.949365,10595.000000,23864.0,23864.0,5.133209,110.567471,543.356569,595.0,597.0,600.0,1,PRLP,0.995000,0.556026,0,1
1,8613.954540,11658.784607,11658.784607,23148.0,23148.0,3.369164,22.200490,585.030410,595.0,596.0,600.0,1,PRLP,0.993333,0.496337,0,1
2,7997.354881,10377.301905,10377.301905,18229.0,18099.0,6.372723,33.418343,578.041435,595.0,596.0,600.0,1,PRLP,0.993333,0.426637,0,1
3,8247.942960,10654.001951,10654.001951,24711.0,24711.0,6.817521,63.228352,560.096522,595.0,597.0,600.0,1,PRLP,0.995000,0.568856,0,1
4,8158.691420,11480.809758,11480.809758,22151.0,22151.0,4.503026,41.085310,567.056812,595.0,596.0,600.0,1,PRLP,0.993333,0.481702,0,1
5,8743.310877,10712.874356,10712.874356,18072.0,17044.0,3.068112,29.984851,575.030260,595.0,596.0,600.0,1,PRLP,0.993333,0.371458,0,1
6,7629.702060,10571.694249,10571.694249,20235.0,16294.0,4.516528,23.513340,583.121127,595.0,595.0,600.0,1,PRLP,0.991667,0.351191,0,1
7,8191.992722,10278.458598,10278.458598,21124.0,21124.0,4.954860,55.283620,566.214191,595.0,597.0,600.0,1,PRLP,0.995000,0.513423,0,1
8,8552.621598,10473.115817,10474.000000,20993.0,20348.0,3.572071,26.283147,580.121972,595.0,596.0,600.0,1,PRLP,0.993333,0.485257,0,1
9,8413.016960,11652.893314,11652.893314,23520.0,23520.0,3.018316,56.731826,566.125579,595.0,597.0,600.0,1,PRLP,0.995000,0.504554,0,1


In [70]:
baseline_dfs['bnd_series_1']

,lpBound,rootDualBound,dualBound,heuristicPrimalBound,primalBound,heuristicTime,rootDualBoundTime,terminationTime,maxTerminationTime,completionTime,maxCompletionTime,usePreprocessing,reltime,gap,nofeas,batch
0,7830.277000,10470.421275,10470.421275,22960.0,17039.0,2.889270,9.095731,600.062999,600.0,601.0,600.0,1,1.001667,0.385503,0,1
1,8613.708184,11592.451451,11592.451451,19922.0,19503.0,3.103909,9.208586,600.030883,600.0,601.0,600.0,1,1.001667,0.405607,0,1
2,7997.125029,10430.146743,10430.146743,21236.0,20389.0,3.029805,9.574398,600.042346,600.0,600.0,600.0,1,1.000000,0.488442,0,1
3,7993.869636,10674.995529,10674.995529,21415.0,21415.0,2.595786,9.948449,600.018932,600.0,601.0,600.0,1,1.001667,0.501518,0,1
4,8158.498294,11493.704638,11493.704638,22940.0,18050.0,2.817163,8.812289,600.028171,600.0,601.0,600.0,1,1.001667,0.363230,0,1
5,8428.339235,10665.425056,10861.941365,17980.0,15039.0,1.803965,4.759056,600.046444,600.0,600.0,600.0,1,1.000000,0.277748,0,1
6,7629.433118,10534.083470,10534.083470,20510.0,16596.0,4.221110,8.238935,600.116084,600.0,600.0,600.0,1,1.000000,0.365264,0,1
7,8191.923029,10308.737334,10308.737334,15582.0,15582.0,4.336580,10.859625,600.029141,600.0,601.0,600.0,1,1.001667,0.338420,0,1
8,8503.342059,10609.219912,10610.000000,21213.0,20483.0,2.629793,6.671688,600.108063,600.0,601.0,600.0,1,1.001667,0.482009,0,1
9,8288.504071,11669.276960,11669.276960,23352.0,23352.0,2.939050,9.199898,600.024263,600.0,600.0,600.0,1,1.000000,0.500288,0,1
